# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [2]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [3]:
# your code here
orders = pd.read_csv("data/Orders/orders.csv")
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [4]:
# your code here
# Orders are grouped by CustomerID and Country, then sum amount_spent per group
grouped_data = orders.groupby(["CustomerID", "Country"])[["amount_spent"]].sum()

# Calculate the 95th percentile threshold for VIP customers
VIP_sum = grouped_data["amount_spent"].quantile(0.95)
print(f"VIP sum: {VIP_sum}")

# Select VIP customers: those who spent equal to or more than the 95th percentile
VIP_customers = grouped_data[grouped_data["amount_spent"] >= VIP_sum].reset_index()

# Calculate the 75th percentile threshold for preferred customers
preferred_customer_sum = grouped_data["amount_spent"].quantile(0.75)
print(f"preferred_customer_sum: {preferred_customer_sum}")

# Select preferred customers: those who spent between the 75th and 95th percentiles
preferred_customers = grouped_data[
    (grouped_data["amount_spent"] >= preferred_customer_sum) &
    (grouped_data["amount_spent"] < VIP_sum)
].reset_index()

# Label each order with the appropriate customer type
# - "VIP" if the CustomerID is in VIP_customers
# - "preferred" if in preferred_customers
# - "regular" otherwise
orders["Customer Type"] = np.where(
    orders["CustomerID"].isin(VIP_customers["CustomerID"]),
    "VIP",
    np.where(
        orders["CustomerID"].isin(preferred_customers["CustomerID"]),
        "preferred",
        "regular"
    )
)

orders

VIP sum: 5774.277999999996
preferred_customer_sum: 1661.195


,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent,Customer Type
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,preferred
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,preferred
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,preferred
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,preferred
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,preferred
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397919,541904,581587,22613,2011,12,5,12,pack of 20 spaceboy napkins,12,2011-12-09 12:50:00,0.85,12680,France,10.20,regular
397920,541905,581587,22899,2011,12,5,12,children's apron dolly girl,6,2011-12-09 12:50:00,2.10,12680,France,12.60,regular
397921,541906,581587,23254,2011,12,5,12,childrens cutlery dolly girl,4,2011-12-09 12:50:00,4.15,12680,France,16.60,regular
397922,541907,581587,23255,2011,12,5,12,childrens cutlery circus parade,4,2011-12-09 12:50:00,4.15,12680,France,16.60,regular


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [5]:
# your code here
sum_of_customer = orders.groupby(["Country", "Customer Type", "CustomerID"])[["amount_spent"]].sum().reset_index()
vip_countries = sum_of_customer.groupby(["Country", "Customer Type"])[["CustomerID"]].count().reset_index()
vip_country_count = vip_countries[vip_countries["Customer Type"] == "VIP"].sort_values("CustomerID", ascending=False)
vip_country_count
# Check out the U.K. with 178 VIP Customers! -> this is the highest value

,Country,Customer Type,CustomerID
71,United Kingdom,VIP,178
30,Germany,VIP,11
27,France,VIP,9
66,Switzerland,VIP,3
60,Spain,VIP,2
54,Portugal,VIP,2
40,Japan,VIP,2
21,EIRE,VIP,2
24,Finland,VIP,1
11,Channel Islands,VIP,1


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [6]:
# your code here
VIP_or_preferred = orders[(orders["Customer Type"] == "VIP") | (orders["Customer Type"] == "preferred")]

sum_of_customers_vop = VIP_or_preferred.groupby(["Country", "CustomerID"])[["amount_spent"]].sum().reset_index()
countries_count_vop = sum_of_customers_vop.groupby(["Country"])[["CustomerID"]].count().reset_index()
countries_count_vop.sort_values("CustomerID", ascending=False)

# Again, the United Kingdom has the highest count of VIP + preferred Customers

,Country,CustomerID
26,United Kingdom,934
10,Germany,39
9,France,29
2,Belgium,12
25,Switzerland,9
23,Spain,9
21,Portugal,7
19,Norway,7
14,Italy,5
8,Finland,5
